**Analysis of Presidential speech and election data**

This notebook scrapes [The American Presidency Project](http://www.presidency.ucsb.edu) and downloads the campagin speeches of all 2016 presidential candidates.  It then builds a markov chain out each president's data capable of generating sentences in the style of their campaign speeches. 

In [4]:
import pandas as pd
import numpy as np
import requests
from lxml import html
from bs4 import BeautifulSoup
import markovify
import os.path
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import re

In [5]:

def getCandidateSpeechLinks(url):
    allCandidatePage = requests.get(url)
    allCandidatePageSoup = BeautifulSoup(allCandidatePage.text, 'lxml')
    links={}
    table = allCandidatePageSoup.find('table', width=680)
    for area in table.findAll('td', class_='doctext'):
        for a in area.findAll('a'):
            if ('campaign' in a.text.lower()):
                links[area.find('span', class_='roman').text] = a['href']
    return links

def scrapeCampaignSpeechesToFile(url, path):
    allSpeechPages = requests.get(url)
    allSpeechSoup=BeautifulSoup(allSpeechPages.text, 'lxml')
    root = 'http://www.presidency.ucsb.edu/'
    table = allSpeechSoup.find('table', width=700)
    links = []
    for link in table.findAll('a'):
        if('interview' not in link.text.lower()):
            links.append(root+(link['href'])[3:])

    speechPages = [requests.get(link , 'lxml')for link in links]
    speechesSoup = [BeautifulSoup(speechPage.text, 'lxml') for speechPage in speechPages]

    with open(path, "w+", encoding='utf-8') as outFile:
        outFile.seek(0)
        for i,speech in enumerate(speechesSoup):            
            text = speechesSoup[i].find('span', class_='displaytext').text.replace('.','. ')
            text = re.sub('\[[a-zA-Z]*\]', ' ', text)
            text = re.sub('[A-Z]+ [A-Z]+:', ' ', text)
            text = re.sub('\w+:', ' ', text)
            text = re.sub(r'[^\x00-\x7F]+',' ', text)
            
            outFile.write(text +'\n')


def campaignLinkToFiles(url, year):
    
    dataFolder = './Campaign Speeches/'+ str(year) +'/'
    
    if not os.path.exists(dataFolder):
        os.makedirs(dataFolder)
    
    #Create the dictionary of each candidate's name and link to their campaign speech page    
    campaignSpeechLinkDict = getCandidateSpeechLinks(url)
    
    root = 'http://www.presidency.ucsb.edu/'
    new=0
    existing=0
    newpaths=[]
    #Loops through the campagin speech links, puts each candidate's campagin speeches into individual files
    for name, url in campaignSpeechLinkDict.items():
        path = dataFolder + name.replace(' ', '-') + '.txt'
        if not os.path.isfile(path):
            new+=1
            newpaths.append(path)
            scrapeCampaignSpeechesToFile(root + url, path)
        else:
            existing+=1
    
    
    
    #print some statistics
    print(str(existing), ' files already existed, ignoring.')
    print(str(new), ' files created successfully:')
    for p in newpaths:
        print(p)

In [7]:
campaignLinkToFiles('http://www.presidency.ucsb.edu/2016_election.php', 2016)

0  files already existed, ignoring.
21  files created successfully:
./Campaign Speeches/2016/Rick-Perry.txt
./Campaign Speeches/2016/Bobby-Jindal.txt
./Campaign Speeches/2016/Hillary-Clinton-.txt
./Campaign Speeches/2016/Jeb-Bush.txt
./Campaign Speeches/2016/George-Pataki.txt
./Campaign Speeches/2016/Marco-Rubio.txt
./Campaign Speeches/2016/Carly-Fiorina.txt
./Campaign Speeches/2016/Rand-Paul.txt
./Campaign Speeches/2016/Scott-Walker.txt
./Campaign Speeches/2016/Chris-Christie.txt
./Campaign Speeches/2016/John-Kasich.txt
./Campaign Speeches/2016/Mike-Huckabee.txt
./Campaign Speeches/2016/Bernie-Sanders.txt
./Campaign Speeches/2016/Rick-Santorum.txt
./Campaign Speeches/2016/Lincoln-Chafee.txt
./Campaign Speeches/2016/Ted-Cruz.txt
./Campaign Speeches/2016/Donald-Trump.txt
./Campaign Speeches/2016/Lindsey-Graham.txt
./Campaign Speeches/2016/Jim-Webb.txt
./Campaign Speeches/2016/Martin-O'Malley.txt
./Campaign Speeches/2016/Ben-Carson.txt


In [ ]:
for name,bot in bots2016.items():
    print('\n' + name + ': ')
    for i in range(10):
        print(bot.make_short_sentence(max_chars=140))

In [ ]:
for i in range(10):
    print(bots2016['Donald Trump'].make_sentence_with_start(beginning='Crooked'))

In [ ]:
bots2008 = campaignLinkToBots('http://www.presidency.ucsb.edu/2008_election.php', 2008)

for name,bot in bots2008.items():
    print('\n' + name + ': ')
    for i in range(10):
        print(bot.make_short_sentence(max_chars=140))